# Notebook 02: Building a Simple Agent with Tools

## 🎯 What is This Notebook About?

Welcome! In this notebook, we'll build **real autonomous agents** with tools using LlamaStack. This is where the magic happens - we're moving from concepts to actual working agents!

**What we'll learn:**
1. How to create a custom Wikipedia search tool (no API key needed - perfect for learning!)
2. How to create sessions and execute turns (the agent's conversation flow)
3. How to see tool calls using AgentEventLogger (watch the agent think and act!)
4. How to create custom (client-side) tools (build your own tools!)
5. How to use custom tools with agents (put it all together)

**Why this matters:**
- This is where agents go from theory to practice
- You'll see agents actually reasoning and taking actions
- You'll learn to build tools that agents can use
- This is the foundation for building production-ready agents

---

## 💼 How This Applies to IT Operations

**The problem:** You need agents that can actually interact with your IT infrastructure - check service status, restart services, scale resources, etc. But how do you give agents these capabilities?

**The solution:** Custom tools! You build Python functions that agents can call. The agent uses its reasoning to decide when to use each tool, and you get to watch it happen in real-time.

**Real-world impact:**
- **Build custom tools** for your specific infrastructure (check your monitoring system, restart your services, etc.)
- **Watch agents reason** about problems and decide which tools to use
- **See tool execution** in real-time with AgentEventLogger (🤔 for thinking, 🔧 for tool calls)
- **Create production-ready agents** that can actually manage your IT operations

**The fun part:** We'll start with Wikipedia (easy to understand) and then build IT operations tools. You'll see the agent check service status, detect problems, and fix them - all autonomously!

---

## ⚙️ Prerequisites

Before starting this notebook, make sure you have:
- ✅ Completed Notebook 01: Introduction to Autonomous Agents
- ✅ LlamaStack server running (see Module README for setup)
- ✅ Ollama running with `llama3.2:3b` model
- ✅ Python environment with dependencies installed
- ✅ `requests` library installed (`pip install requests`)

**The fun part:** We'll start with Wikipedia (no API key needed!) and then build IT operations tools. You'll see agents actually using tools in real-time!


In [ ]:
# Import required libraries
import os
from llama_stack_client import LlamaStackClient, Agent, AgentEventLogger
from termcolor import cprint

# Configuration
llamastack_url = os.getenv("LLAMA_STACK_URL", "http://localhost:8321")
model = os.getenv("LLAMA_MODEL", "ollama/llama3.2:3b")

print(f"📡 LlamaStack URL: {llamastack_url}")
print(f"🤖 Model: {model}")

# Initialize LlamaStack client
client = LlamaStackClient(base_url=llamastack_url)

# Verify connection
try:
    models = client.models.list()
    print(f"\n✅ Connected to LlamaStack")
    print(f"   Available models: {len(models)}")
except Exception as e:
    print(f"\n❌ Cannot connect to LlamaStack: {e}")
    print("   Please ensure LlamaStack is running:")
    print("   python scripts/start_llama_stack.py")
    raise


---

## Part 1: Create an Agent with Custom Wikipedia Search

**What we're doing:** Creating a **custom Wikipedia search tool** and giving it to an agent.

**Why:** Wikipedia is perfect for learning because:
- ✅ No API key required (free and easy!)
- ✅ Runs client-side (in your Python process - no external dependencies)
- ✅ Shows how to create custom tools (the pattern you'll use for everything)
- ✅ Perfect for factual information and general knowledge

**The fun part:** We'll create a `wikipedia_search` function and pass it to the agent. The agent will automatically detect it as a tool based on the function's docstring! No complex configuration needed - just write a function with a good docstring, and the agent figures it out.


In [ ]:
# Step 1: Create a custom Wikipedia search tool
# Wikipedia doesn't require an API key - perfect for learning!
print("=" * 60)
print("Creating Custom Wikipedia Search Tool")
print("=" * 60)

# Import requests for API calls
import requests

def wikipedia_search(query: str, max_results: int = 3) -> str:
    """
    Search Wikipedia for information.
    
    This tool searches Wikipedia for factual information. Use it when you need
    general knowledge, definitions, or information about well-known topics.
    
    :param query: The search query to look up (e.g., "Python programming", "Artificial Intelligence")
    :param max_results: Maximum number of results to return (default: 3)
    :return: A formatted string with Wikipedia search results
    """
    try:
        import requests
        
        # First, try the search API (more flexible for general queries)
        search_url = "https://en.wikipedia.org/api/rest_v1/page/search"
        params = {"q": query, "limit": max_results}
        search_response = requests.get(search_url, params=params, timeout=5)
        
        if search_response.status_code == 200:
            results = search_response.json().get("pages", [])
            if results:
                formatted = f"Wikipedia search results for '{query}':\n\n"
                for j, page in enumerate(results[:max_results], 1):
                    title = page.get("title", "No title")
                    snippet = page.get("snippet", "No snippet")
                    # Try to get the full summary for the first result
                    if j == 1:
                        try:
                            page_url = f"https://en.wikipedia.org/api/rest_v1/page/summary/{title.replace(' ', '_')}"
                            page_response = requests.get(page_url, timeout=5)
                            if page_response.status_code == 200:
                                page_data = page_response.json()
                                extract = page_data.get("extract", snippet)
                                formatted += f"{j}. {title}\n"
                                formatted += f"   {extract[:400]}...\n\n"
                            else:
                                formatted += f"{j}. {title}\n"
                                formatted += f"   {snippet[:300]}...\n\n"
                        except:
                            formatted += f"{j}. {title}\n"
                            formatted += f"   {snippet[:300]}...\n\n"
                    else:
                        formatted += f"{j}. {title}\n"
                        formatted += f"   {snippet[:300]}...\n\n"
                return formatted.strip()
        
        # Fallback: try direct page access
        page_url = "https://en.wikipedia.org/api/rest_v1/page/summary/" + query.replace(" ", "_")
        response = requests.get(page_url, timeout=5)
        
        if response.status_code == 200:
            data = response.json()
            title = data.get("title", query)
            extract = data.get("extract", "No summary available")
            page_url_desktop = data.get("content_urls", {}).get("desktop", {}).get("page", "")
            
            result = f"Wikipedia: {title}\n"
            result += f"URL: {page_url_desktop}\n"
            result += f"Summary: {extract[:500]}...\n"
            return result
            
        return f"No Wikipedia results found for: {query}"
            
    except Exception as e:
        return f"Error searching Wikipedia: {str(e)}"
print("\n✅ Custom wikipedia_search tool created!")
print("   Provider: Wikipedia (no API key required)")
print("   Function: wikipedia_search(query: str, max_results: int = 3)")

# Step 2: Create an agent with our custom wikipedia_search tool
print("\n" + "=" * 60)
print("Creating Agent with Custom wikipedia_search Tool")
print("=" * 60)

agent = Agent(
    client,
    model=model,
    instructions="You are a helpful assistant. Use the wikipedia_search tool to help answer questions that require factual information or general knowledge.",
    tools=[wikipedia_search],  # Pass the function directly - Agent will detect it via docstring!
)

print("\n✅ Agent created successfully!")
print(f"   Tool: wikipedia_search (custom, using Wikipedia)")
print(f"   Model: {model}")
print(f"   Execution: Client-side (runs in this Python process)")


In [ ]:
# Step 2: Create a session
# Sessions maintain conversation context
print("\n" + "=" * 60)
print("Creating Session")
print("=" * 60)

session_id = agent.create_session("test-session")
print(f"\n✅ Session created!")
print(f"   Session ID: {session_id}")
print(f"\n💡 Sessions maintain conversation history and context.")

In [ ]:
# Step 3: Execute a turn (give the agent a task)
# A turn is one interaction with the agent
print("\n" + "=" * 60)
print("Executing Turn with Task")
print("=" * 60)

task = "What is Python programming? Use wikipedia_search to find information."

print(f"\n📋 Task: {task}")
print(f"\n🔄 Agent execution (using AgentEventLogger):")
print("-" * 60)

cprint(f"User> {task}", "green")

# Create turn with streaming to see tool calls in real-time
# Streaming shows tool calls as they happen!
response = agent.create_turn(
    messages=[
        {
            "role": "user",
            "content": task,
        }
    ],
    session_id=session_id,
    stream=True,  # Enable streaming to see tool calls
)

# AgentEventLogger shows tool calls visually!
# 🤔 = Agent reasoning
# 🔧 = Tool being executed
tool_calls_found = False
for log in AgentEventLogger().log(response):
    print(log, end="", flush=True)
    if "🔧" in log or "tool" in log.lower():
        tool_calls_found = True

print(f"\n\n{'=' * 60}")
if tool_calls_found:
    print("✅ Turn completed! (Tools were used)")
else:
    print("⚠️  Turn completed, but no tool calls detected in output")
    print("   The agent may have answered without searching, or tool calls")
    print("   may be logged differently.")
print(f"{'=' * 60}")

# Inspect the turn - check if tools were called
print("\n" + "=" * 60)
print("Inspecting Turn Response")
print("=" * 60)

# The AgentEventLogger output above already shows tool usage
# If you see 🔧 in the output, the tool was used successfully!

print("\n✅ Turn completed!")
print("\n💡 Tip: Look for these indicators in the output above:")
print("   🤔 = Agent reasoning")
print("   📞 = Function call detected")
print("   🔧 = Tool being executed")
print("   → = Tool result")
print("\nIf you see 🔧, the tool was used successfully!")

# Note: The Agent class manages sessions internally
# To inspect session details, you would need to use the client API directly
# with the agent_id, but the Agent class doesn't expose this easily.
# The AgentEventLogger output is sufficient to see tool usage.


---

## Part 2: Create Custom IT Operations Tools

**What we're doing:** Creating **custom IT operations tools** - Python functions that agents can use to manage infrastructure.

**Why:** This is where it gets real! We'll build tools that check service status, restart services, and get system overviews - the kind of tools you'd use in production.

**Key Points:**
- Custom tools are Python functions with **docstrings** (the docstring is critical - the LLM reads it!)
- Docstrings describe the tool to the LLM (what it does, when to use it, what parameters it needs)
- Tools are passed directly to the Agent (no complex configuration)
- Agent automatically detects and uses them (based on the docstring - magic!)

**The fun part:** You'll see the agent reason about which tool to use. It might check status first, then decide to restart a service if needed. Watch it think!

In [ ]:
# Step 1: Create a simple simulated IT environment
class SimpleITEnvironment:
    """Simple simulated IT environment for our custom tools."""
    def __init__(self):
        self.services = {
            "web-server": {"status": "online", "cpu": 45, "memory": 60},
            "database": {"status": "online", "cpu": 30, "memory": 50},
            "cache-service": {"status": "degraded", "cpu": 85, "memory": 90},
        }
    
    def get_service_status(self, service_name: str) -> dict:
        """Get service status."""
        return self.services.get(service_name, {"status": "not_found"})
    
    def restart_service(self, service_name: str) -> str:
        """Restart a service."""
        if service_name in self.services:
            self.services[service_name]["status"] = "online"
            self.services[service_name]["cpu"] = 20
            self.services[service_name]["memory"] = 30
            return f"Service {service_name} restarted successfully"
        return f"Service {service_name} not found"


# Initialize environment
env = SimpleITEnvironment()
print("✅ Created simulated IT environment")
print(f"   Services: {list(env.services.keys())}")

In [ ]:
# Step 2: Define custom tools as Python functions with docstrings
# The docstrings are CRITICAL - the LLM uses them to understand the tools!

def check_service_status(service_name: str) -> str:
    """
    Check the status of an IT service.
    
    Returns information about service health including CPU usage, memory usage, and current status.
    Use this to monitor service health and detect issues.
    
    :param service_name: The name of the service to check (e.g., 'web-server', 'database', 'cache-service')
    :return: A string describing the service status, CPU usage, and memory usage
    """
    status = env.get_service_status(service_name)
    if status.get("status") == "not_found":
        return f"Service '{service_name}' not found"
    
    return (
        f"Service: {service_name}\n"
        f"Status: {status['status']}\n"
        f"CPU Usage: {status['cpu']}%\n"
        f"Memory Usage: {status['memory']}%"
    )


def restart_service(service_name: str) -> str:
    """
    Restart an IT service that is not working properly.
    
    Use this when a service has failed or is in a degraded state.
    This will stop and start the service, which may cause brief downtime.
    
    :param service_name: The name of the service to restart (e.g., 'web-server', 'database')
    :return: A string confirming the restart operation
    """
    return env.restart_service(service_name)


def get_all_services() -> str:
    """
    Get the status of all services in the environment.
    
    Use this to get an overview of the entire system health.
    Returns a list of all services with their current status and metrics.
    
    :return: A string listing all services and their status
    """
    result = "All Services Status:\n"
    for name, status in env.services.items():
        result += f"\n{name}:\n"
        result += f"  Status: {status['status']}\n"
        result += f"  CPU: {status['cpu']}%\n"
        result += f"  Memory: {status['memory']}%\n"
    return result


print("\n✅ Custom tools defined:")
print("   - check_service_status(service_name: str)")
print("   - restart_service(service_name: str)")
print("   - get_all_services()")
print("\n💡 Key Points:")
print("   - Tools are Python functions with docstrings")
print("   - Docstrings describe what the tool does")
print("   - :param describes parameters")
print("   - The LLM reads these docstrings to understand when to use each tool")

In [ ]:
# Step 3: Create an agent with custom tools
# Pass the functions directly - Agent will detect them via docstrings!
print("\n" + "=" * 60)
print("Creating Agent with Custom Tools")
print("=" * 60)

custom_agent = Agent(
    client,
    model=model,
    instructions=(
        "You are an IT operations agent. "
        "Use the available tools to monitor and manage IT services. "
        "Always check service status before taking actions. "
        "Provide clear summaries of what you did."
    ),
    tools=[
        check_service_status,
        restart_service,
        get_all_services,
    ],
)

print("\n✅ Agent created with custom tools!")
print("   Tools: check_service_status, restart_service, get_all_services")
print("   Note: Tools run client-side (in this Python process)")

In [ ]:
# Step 4: Test the agent with custom tools
print("\n" + "=" * 60)
print("Testing Agent with Custom Tools")
print("=" * 60)

# Create a new session for custom tools
custom_session_id = custom_agent.create_session("test-custom-tools")
print(f"\n📝 Created session: {custom_session_id}")

# Test tasks
test_tasks = [
    "Check the status of all services",
    "Check the status of cache-service and restart it if it's degraded",
]

for task in test_tasks:
    print(f"\n{'=' * 60}")
    print(f"Task: {task}")
    print("=" * 60)
    
    cprint(f"User> {task}", "green")
    
    response = custom_agent.create_turn(
        messages=[{"role": "user", "content": task}],
        session_id=custom_session_id,
    )
    
    # AgentEventLogger shows tool calls!
    for log in AgentEventLogger().log(response):
        print(log, end="", flush=True)
    
    print()  # New line after each response

print(f"\n{'=' * 60}")
print("✅ All tests completed!")
print(f"{'=' * 60}")

---

## 🎓 Key Takeaways

**What we learned:**

1. **Custom Tools** are Python functions with docstrings - the agent detects them automatically (no complex configuration!)
2. **Wikipedia Search** is a great example - free, no API key, perfect for learning the pattern
3. **Agent Class** simplifies agent creation and tool management - just pass functions and go!
4. **AgentEventLogger** shows tool calls visually - watch the agent think (🤔) and act (🔧) in real-time
5. **Sessions** maintain conversation context - agents remember what happened before

**The big picture:**
- Tools are the agent's "hands" - they let agents take actions, not just think
- Docstrings are critical - the LLM reads them to understand when to use each tool
- Client-side tools run in your Python process - fast, secure, no external dependencies
- You can build any tool you need - Wikipedia, IT operations, databases, APIs, anything!

**For IT operations:**
- Build tools for your specific infrastructure (monitoring systems, service management, etc.)
- Watch agents reason about problems and choose the right tools
- See agents execute multi-step operations (check status → detect problem → fix it)
- Create production-ready agents that can actually manage your systems

---

## 🚀 Next Steps

**Ready for more?** In **Notebook 03**, we'll explore:
- **LlamaStack Core Features** - Chat and RAG (Retrieval Augmented Generation)
- **When to use each feature** - Understanding which tool fits which problem
- **Building more powerful agents** - Combining features for better results

**The fun part:** You'll learn how to give agents access to knowledge bases (RAG) so they can answer questions about your specific infrastructure!

---

**Ready?** Let's move to Notebook 03: LlamaStack Core Features! 🚀
